In [1]:
from models import reswhy
import joblib
import numpy as np

%load_ext autoreload
%autoreload 2

/Users/aliramlaoui/anaconda3/envs/recom-articles/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You passed along `num_labels=1200` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
All TF 2.0 model weights were used when initializing DistilBertForMaskedLM.

Some weights of DistilBertForMaskedLM were not initialized from the TF 2.0 model and are newly initialized: ['vocab_projector.weight', 'vocab_projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Bert

In [109]:
ranking_bm25 = joblib.load("./rankings/ranking_BM25.pkl")
ranking_dbertft = joblib.load("./rankings/ranking_dbertft.pkl")

users_keys = list(ranking_dbertft.keys())

In [110]:
for user in users_keys:
    print(ranking_bm25[user])
    break

[(0, 10527.36901473999, 1), (1, 7833.827178955078, 1), (2, 4969.663930892944, 1), (3, 3596.261800765991, 1), (4, 3051.002191543579, 1), (5, 7083.260164260864, 1), (6, 11016.811786651611, 1), (7, 9635.719787597656, 1), (8, 8616.325408935547, 1), (9, 7312.386749267578, 1), (10, 6070.244110107422, 0), (11, 10666.476259231567, 0), (12, 4700.507340431213, 0), (13, 2685.1768112182617, 0), (14, 7385.395896911621, 0), (15, 2742.937168121338, 0), (16, 6830.004614830017, 0), (17, 13688.036586761475, 0), (18, 13826.198623657227, 0), (19, 6340.41951751709, 0), (20, 5750.232782363892, 0), (21, 8531.248092651367, 0), (22, 6008.10725402832, 0), (23, 7990.778568267822, 0), (24, 3788.490291595459, 0), (25, 6920.738466262817, 0), (26, 12300.149467468262, 0), (27, 6799.1229820251465, 0), (28, 6499.231369018555, 0), (29, 6399.728811264038, 0), (30, 10511.182725906372, 0), (31, 5831.746806144714, 0), (32, 3257.706117630005, 0), (33, 5194.469060897827, 0), (34, 4514.319894790649, 0), (35, 10143.08584022522,

In [126]:
from scores import get_ndcg_reswhy, grid_search_reswhy

best_params_reswhy = grid_search_reswhy(ranking_bm25, ranking_dbertft)
best_alpha1, best_alpha2, best_beta, best_weight = best_params['alpha1'], best_params['alpha2'], best_params['beta'], best_params['weight']
ranking_reswhy = get_ndcg_reswhy(ranking_bm25, ranking_dbertft, [best_alpha1, best_alpha2], best_beta, best_weight, CV=False, return_ranking=True)


In [127]:
def MRR(scores):
    scores.sort(key=lambda x:-x[1])
    for k in range(len(scores)):
        if scores[k][2]==1:
            return 1/(k+1)

In [131]:
ranking = ranking_reswhy
mrr_total = 0
for user in ranking.keys():
    mrr_total += MRR(ranking[user])
print(mrr_total/len(ranking.keys()))

0.4525596109430828
